In [ ]:
filter_names_viking = ['paranal.vircam.Z', 'paranal.vircam.Y', 'paranal.vircam.J', 'paranal.vircam.H', 'paranal.vircam.Ks']
filter_names_des = 
filter_names_catwise = ['wise.W1', 'wise.W2']

In [1]:
import numpy as np

## Add custom DES filters

### Normalize the transmission

In [2]:
g_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.g.dat'
i_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.i.dat'
r_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.r.dat'
z_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.z.dat'
y_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.Y.dat'

In [3]:
def normalize_filter_data(input_filename, output_filename):
    """
    Normalizes the second column (transmission values) of a .dat file.

    Parameters:
    - input_filename (str): Path to the input .dat file.
    - output_filename (str): Path where the normalized data will be saved.

    The input file should contain two columns: wavelength (first column) and transmission (second column).
    """
    # Initialize lists to store data
    lambda_values = []
    transmission_values = []

    # Read the data from the .dat file
    with open(input_filename, 'r') as f:
        for line in f:
            # Skip comments or empty lines
            if line.startswith('#') or line.strip() == '':
                continue
            
            # Split the line into wavelength and transmission
            columns = line.split()
            lambda_values.append(float(columns[0]))  # Wavelength (first column)
            transmission_values.append(float(columns[1]))  # Transmission (second column)

    # Convert the lists to numpy arrays for easier processing
    lambda_values = np.array(lambda_values)
    transmission_values = np.array(transmission_values)

    # Normalize the transmission values (second column)
    max_transmission = np.max(transmission_values)
    normalized_transmission = transmission_values / max_transmission

    # Save the normalized data to the output .dat file
    with open(output_filename, 'w') as f:
        # Optionally add a header
        f.write("# Wavelength (Å) and Normalized Transmission Values\n")
        
        # Write the wavelength and the normalized transmission
        for lambda_val, norm_trans in zip(lambda_values, normalized_transmission):
            f.write(f"{lambda_val} {norm_trans}\n")

    print(f"Normalization complete. Output saved to {output_filename}.")

In [4]:
normalize_filter_data(g_filter_path, g_filter_path)
normalize_filter_data(i_filter_path, i_filter_path)
normalize_filter_data(r_filter_path, r_filter_path)
normalize_filter_data(y_filter_path, y_filter_path)
normalize_filter_data(z_filter_path, z_filter_path)


Normalization complete. Output saved to /data/mfonseca/survey_data/DES_data/CTIO_DECam.g.dat.
Normalization complete. Output saved to /data/mfonseca/survey_data/DES_data/CTIO_DECam.i.dat.
Normalization complete. Output saved to /data/mfonseca/survey_data/DES_data/CTIO_DECam.r.dat.
Normalization complete. Output saved to /data/mfonseca/survey_data/DES_data/CTIO_DECam.Y.dat.
Normalization complete. Output saved to /data/mfonseca/survey_data/DES_data/CTIO_DECam.z.dat.
